In [2]:
import tkinter as tk
from tkinter import ttk
import time 
import threading
import cv2
import numpy as np
import dlib
from math import hypot
import time
import pandas as pd
from fast_autocomplete import AutoComplete

In [3]:
df = pd.read_csv("data.csv") 
words = {}
empty = {}
for index, row in df.iterrows():
    if str(type(row['word'])) != "<class 'str'>":
        continue
    if len(row['word']) <= 4:
        continue
    words[row['word']] = empty
    if len(words) > 20000:
        break
    
autocomplete = AutoComplete(words = words)

In [4]:
root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

In [5]:
# highlights col based on col_num
def highlightCol(cols, colnum):
    for i in range(len(cols)):
        selectedCol = cols[i]
        for j in range(len(selectedCol)):
            selectedBtn = selectedCol[j]
            selectedBtn["bg"] = "#f7f7f7"
    
    currCol = cols[colnum]
    for i in range(len(currCol)):
        currBtn = currCol[i]
        currBtn["bg"] = "#5a79a5"
        
# hightlights key based on col num and row num
def highlightBtn(cols, colnum, rownum):
    currBtn = cols[colnum][rownum]
    
    for i in range(len(cols)):
        selectedCol = cols[i]
        for j in range(len(selectedCol)):
            selectedBtn = selectedCol[j]
            selectedBtn["bg"] = "#f7f7f7"
    
    currBtn["bg"] = "#5a79a5"   
    
def get_letter_key(cols, column, rownum):
    currBtn = cols[column][rownum]
    return currBtn["text"]

In [6]:
# updates the suggested list words 
def update(text_entry, cols):
    currCol = cols[len(cols)-1]
    currTxt = text_entry.get()
    
    lis = list(currTxt.split(" "))
    length = len(lis)
     
    lastWord = lis[length-1]
    listOfWords = autocomplete.search(word = lastWord , max_cost=1, size=6)
    
    i = 0
    for btn in currCol:
        if i >= len(listOfWords):
            btn["text"] = ""
        else:
            btn["text"] = listOfWords[i]
        i += 1

In [7]:
# prints letter using row_num and col_num
def clickBtn(text_entry, cols, colnum, rownum):
    extra = {"backspace", "space", "enter", "clear", "close"}
    btn = cols[colnum][rownum]
    currBtn = btn
    if colnum == len(cols)-1:
        index = text_entry.get().rfind(" ") + 1
        text_entry.delete(index, "end")
        text_entry.insert(len(text_entry.get()), currBtn["text"]) 
    elif btn["text"] in extra:
        currChar = currBtn["text"]
        if currChar == "backspace":
            if len(text_entry.get()) == 0:
                temp = 0
            else:
                text_entry.delete(len(text_entry.get())-1,"end")
        elif currChar == "space":
            text_entry.insert(len(text_entry.get()), ' ')
        elif currChar == "enter":
            text_entry.insert(len(text_entry.get()), '\n')
        elif currChar == "clear":
            text_entry.delete(len(text_entry.get()),"end")
        elif currChar == "close":
            window.close()
    else:
        if len(text_entry.get()) == 0:
            text_entry.insert(0,"   ")
        text_entry.insert(len(text_entry.get()), currBtn["text"])
    update(text_entry, cols)
        
        
# prints letter using just button 
def clickThisBtn(text_entry, btn):
    extra = {"backspace", "space", "enter", "clear", "close"}
    if btn["text"] in extra:
        currBtn = btn
        currChar = currBtn["text"]
        if currChar == "backspace":
            if len(text_entry.get()) == 0:
                temp = 0
            else:
                text_entry.delete(len(text_entry.get())-1,"end")
        elif currChar == "space":
            text_entry.insert(len(text_entry.get()), ' ')
        elif currChar == "enter":
            text_entry.insert(len(text_entry.get()), '\n')
        elif currChar == "clear":
            text_entry.delete(len(text_entry.get()),"end")
        elif currChar == "close":
            window.close()
    else:
        if len(text_entry.get()) == 0:
            text_entry.insert(0,"   ")
        text_entry.insert(len(text_entry.get()), btn["text"])

In [8]:
class tkinterApp(tk.Tk):
    def __init__(self, *args, **kwargs): 
        tk.Tk.__init__(self, *args, **kwargs)
        
        container = tk.Frame(self,width = screen_width, height = screen_height)
        self.title("Eye Gaze Controlled Keyboard")
        screen_dimension = str(screen_width) + "x" + str(screen_height)
        self.geometry(screen_dimension)
        
        container.pack(side = "top", fill = "both", expand = True)
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
        
        self.frames = {} 
         
        frame = Main(container, self)
        self.frames[Main] = frame 
        frame.grid(row = 0, column = 0, sticky ="nsew")
        self.show_frame(Main)
        
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
font = cv2.FONT_HERSHEY_PLAIN

In [10]:
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    #hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght
    return ratio


def eyes_contour_points(facial_landmarks):
    left_eye = []
    right_eye = []
    for n in range(36, 42):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        left_eye.append([x, y])
    for n in range(42, 48):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        right_eye.append([x, y])
    left_eye = np.array(left_eye, np.int32)
    right_eye = np.array(right_eye, np.int32)
    return left_eye, right_eye

def get_gaze_ratio(eye_points, facial_landmarks,frame, gray):
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
    # cv2.polylines(frame, [left_eye_region], True, (0, 0, 255), 2)

    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white
    return gaze_ratio

In [12]:
class Main(tk.Frame):
    frames_count_1 = 0
    
    #for if highlights
    key_blink = False
    key_gaze = True
    key_row = False
    
    #count frames
    blinking_frames = 0
    index_letter = 0
    col_letter = 0
    blinking_frames_le = 0
    blinking_frames_ri = 0
    
    gaze_ratio = 0
    blinking_ratio = 0
    
    gaze_sight = ""
    
    #active_constants
    active_blinking = 4
    active_gaze = 15
    frame_limit = 25
    
    text = ""
    
    
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
        self.configure(background = "#dee7ef")
    
        heading_x = (screen_width//2) - 200
        heading_y = 50   
        
        labelHeading = tk.Label(self, text = "Eye Gaze Controlled Keyboard",background = "#dee7ef", font=("Helvetica", 20, 'bold'), fg = '#5a79a5')
        labelHeading.place(x = heading_x, y = heading_y)
        
        x_gaze = (screen_width) - 300
#         entryText_1 = tk.String()
        
        text_entry_1 = tk.Entry(self, font=("Helvetica", 14), background = "#f7f7f7")
        text_entry_1.place(x = x_gaze , y = heading_y , width =250 , height = 50)
        text_entry_1.insert(0, "This is the default text")
        text_entry_1.configure(state='disabled')
        
        text_entry_w = screen_width*0.7
        text_entry_h = screen_height*0.3
        
        text_entry = tk.Entry(self, font=("Helvetica", 18), background = "#f7f7f7")
        text_entry.place(x = screen_width/2 - text_entry_w/2 , y = heading_y + 80, width = text_entry_w, height = text_entry_h)

        keyboard_x = 20 
        keyboard_y = heading_y + text_entry_h + 120 
        
        kw = 10
        
        b1 = tk.Button(self, text = "backspace", width = kw, pady = 5, bg="#f7f7f7",relief = "sunken")
        b1.configure(command = lambda: clickThisBtn(text_entry, b1))
        b1.place(x = keyboard_x,y= keyboard_y)   
        b2 = tk.Button(self, text = "space",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b2.configure(command = lambda: clickThisBtn(text_entry, b2))
        b2.place(x = keyboard_x,y= keyboard_y + 50) 
        b3 = tk.Button(self, text = "enter",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b3.configure(command = lambda: clickThisBtn(text_entry, b3))
        b3.place(x = keyboard_x,y= keyboard_y + 100) 
        b4 = tk.Button(self, text = "clear",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b4.configure(command = lambda: clickThisBtn(text_entry, b4))
        b4.place(x = keyboard_x,y= keyboard_y + 150) 
        b5 = tk.Button(self, text = "close",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b5.configure(command = lambda: clickThisBtn(text_entry, b5))
        b5.place(x = keyboard_x,y= keyboard_y + 200) 
        
        xx = screen_width*0.057
        d =  screen_width*0.057
        
        b6 = tk.Button(self, text = "q",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b6.configure(command = lambda: clickThisBtn(text_entry, b6))
        b6.place(x = keyboard_x + xx,y= keyboard_y ) 
        b7 = tk.Button(self, text = "a",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b7.configure(command = lambda: clickThisBtn(text_entry, b7))   
        b7.place(x = keyboard_x + xx, y = keyboard_y + 50)   
        b8 = tk.Button(self, text = ",",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b8.configure(command = lambda: clickThisBtn(text_entry, b8)) 
        b8.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b9 = tk.Button(self, text = "!",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b9.configure(command = lambda: clickThisBtn(text_entry, b9))
        b9.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b10 = tk.Button(self, text = "<",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b10.configure(command = lambda: clickThisBtn(text_entry, b10))
        b10.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b11 = tk.Button(self, text = "p",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b11.configure(command = lambda: clickThisBtn(text_entry, b11))
        b11.place(x = keyboard_x + xx,y= keyboard_y ) 
        b12 = tk.Button(self, text = "n",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b12.configure(command = lambda: clickThisBtn(text_entry, b12))
        b12.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b13 = tk.Button(self, text = ".",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b13.configure(command = lambda: clickThisBtn(text_entry, b13))
        b13.place(x = keyboard_x + xx, y = keyboard_y+100)   
        b14 = tk.Button(self, text = "\"",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b14.configure(command = lambda: clickThisBtn(text_entry, b14))
        b14.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b15 = tk.Button(self, text = ">",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b15.configure(command = lambda: clickThisBtn(text_entry, b15))
        b15.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b16 = tk.Button(self, text = "y",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b16.configure(command = lambda: clickThisBtn(text_entry, b16))
        b16.place(x = keyboard_x + xx,y= keyboard_y) 
        b17 = tk.Button(self, text = "l",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b17.configure(command = lambda: clickThisBtn(text_entry, b17))
        b17.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b18 = tk.Button(self, text = "j",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b18.configure(command = lambda: clickThisBtn(text_entry, b18))
        b18.place(x = keyboard_x + xx,y= keyboard_y + 100)  
        b19 = tk.Button(self, text = "?",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b19.configure(command = lambda: clickThisBtn(text_entry, b19))
        b19.place(x = keyboard_x + xx, y = keyboard_y + 150)   
        b20 = tk.Button(self, text = "#",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b20.configure(command = lambda: clickThisBtn(text_entry, b20))
        b20.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b21 = tk.Button(self, text = "c",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b21.configure(command = lambda: clickThisBtn(text_entry, b21))
        b21.place(x = keyboard_x + xx,y= keyboard_y) 
        b22 = tk.Button(self, text = "s",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b22.configure(command = lambda: clickThisBtn(text_entry, b22))
        b22.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b23 = tk.Button(self, text = "g",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b23.configure(command = lambda: clickThisBtn(text_entry, b23))
        b23.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b24 = tk.Button(self, text = "@",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b24.configure(command = lambda: clickThisBtn(text_entry, b24))
        b24.place(x = keyboard_x + xx,y= keyboard_y + 150)  
        b25 = tk.Button(self, text = "$",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b25.configure(command = lambda: clickThisBtn(text_entry, b25))
        b25.place(x = keyboard_x + xx, y = keyboard_y + 200) 
        
        xx += d
        
        b26 = tk.Button(self, text = "b",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b26.configure(command = lambda: clickThisBtn(text_entry, b26))
        b26.place(x = keyboard_x + xx,y= keyboard_y ) 
        b27 = tk.Button(self, text = "f",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b27.configure(command = lambda: clickThisBtn(text_entry, b27))
        b27.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b28 = tk.Button(self, text = "e",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b28.configure(command = lambda: clickThisBtn(text_entry, b28))
        b28.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b29 = tk.Button(self, text = "'",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b29.configure(command = lambda: clickThisBtn(text_entry, b29))
        b29.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b30 = tk.Button(self, text = "{",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b30.configure(command = lambda: clickThisBtn(text_entry, b30))
        b30.place(x = keyboard_x + xx,y= keyboard_y + 200)  
        
        xx += d

        b31 = tk.Button(self, text = "7",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b31.configure(command = lambda: clickThisBtn(text_entry, b31))
        b31.place(x = keyboard_x + xx, y = keyboard_y)   
        b32 = tk.Button(self, text = "4",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b32.configure(command = lambda: clickThisBtn(text_entry, b32))
        b32.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b33 = tk.Button(self, text = "1",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b33.configure(command = lambda: clickThisBtn(text_entry, b33))
        b33.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b34 = tk.Button(self, text = "0",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b34.configure(command = lambda: clickThisBtn(text_entry, b34))
        b34.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b35 = tk.Button(self, text = "}",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b35.configure(command = lambda: clickThisBtn(text_entry, b35))
        b35.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b36 = tk.Button(self, text = "8",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b36.configure(command = lambda: clickThisBtn(text_entry, b36))
        b36.place(x = keyboard_x + xx,y= keyboard_y )  
        b37 = tk.Button(self, text = "5",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b37.configure(command = lambda: clickThisBtn(text_entry, b37))
        b37.place(x = keyboard_x + xx, y = keyboard_y + 50)   
        b38 = tk.Button(self, text = "2",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b38.configure(command = lambda: clickThisBtn(text_entry, b38))
        b38.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b39 = tk.Button(self, text = ".",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b39.configure(command = lambda: clickThisBtn(text_entry, b39))
        b39.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b40 = tk.Button(self, text = "[",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b40.configure(command = lambda: clickThisBtn(text_entry, b40))
        b40.place(x = keyboard_x + xx,y= keyboard_y + 200)
        
        xx += d
        
        b41 = tk.Button(self, text = "9",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b41.configure(command = lambda: clickThisBtn(text_entry, b41))
        b41.place(x = keyboard_x + xx,y= keyboard_y ) 
        b42 = tk.Button(self, text = "6",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b42.configure(command = lambda: clickThisBtn(text_entry, b42))
        b42.place(x = keyboard_x + xx,y= keyboard_y + 50)   
        b43 = tk.Button(self, text = "3",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b43.configure(command = lambda: clickThisBtn(text_entry, b43))
        b43.place(x = keyboard_x + xx, y = keyboard_y + 100)   
        b44 = tk.Button(self, text = "enter",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b44.configure(command = lambda: clickThisBtn(text_entry, b44))
        b44.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b45 = tk.Button(self, text = "]",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b45.configure(command = lambda: clickThisBtn(text_entry, b45))
        b45.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b46 = tk.Button(self, text = "*",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b46.configure(command = lambda: clickThisBtn(text_entry, b46))
        b46.place(x = keyboard_x + xx,y= keyboard_y ) 
        b47 = tk.Button(self, text = "/",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b47.configure(command = lambda: clickThisBtn(text_entry, b47))
        b47.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b48 = tk.Button(self, text = "+",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b48.configure(command = lambda: clickThisBtn(text_entry, b48))
        b48.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b49 = tk.Button(self, text = "-",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b49.configure(command = lambda: clickThisBtn(text_entry, b49))
        b49.place(x = keyboard_x + xx, y = keyboard_y + 150)   
        b50 = tk.Button(self, text = "=",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b50.configure(command = lambda: clickThisBtn(text_entry, b50))
        b50.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b51 = tk.Button(self, text = "v",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b51.configure(command = lambda: clickThisBtn(text_entry, b51))
        b51.place(x = keyboard_x + xx,y= keyboard_y ) 
        b52 = tk.Button(self, text = "d",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b52.configure(command = lambda: clickThisBtn(text_entry, b52))
        b52.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b53 = tk.Button(self, text = ";",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b53.configure(command = lambda: clickThisBtn(text_entry, b53))
        b53.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b54 = tk.Button(self, text = ":",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b54.configure(command = lambda: clickThisBtn(text_entry, b54))
        b54.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b55 = tk.Button(self, text = "%",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b55.configure(command = lambda: clickThisBtn(text_entry, b55))
        b55.place(x = keyboard_x + xx, y = keyboard_y + 200)  
        
        xx += d
        
        b56 = tk.Button(self, text = "m",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b56.configure(command = lambda: clickThisBtn(text_entry, b56))
        b56.place(x = keyboard_x + xx,y= keyboard_y) 
        b57 = tk.Button(self, text = "t",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b57.configure(command = lambda: clickThisBtn(text_entry, b57))
        b57.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b58 = tk.Button(self, text = "w",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b58.configure(command = lambda: clickThisBtn(text_entry, b58))
        b58.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b59 = tk.Button(self, text = "^",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b59.configure(command = lambda: clickThisBtn(text_entry, b59))
        b59.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b60 = tk.Button(self, text = "\\",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b60.configure(command = lambda: clickThisBtn(text_entry, b60))
        b60.place(x = keyboard_x + xx,y= keyboard_y + 200)
        
        xx += d
        
        b61 = tk.Button(self, text = "u",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b61.configure(command = lambda: clickThisBtn(text_entry, b61))
        b61.place(x = keyboard_x + xx, y = keyboard_y)   
        b62 = tk.Button(self, text = "h",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b62.configure(command = lambda: clickThisBtn(text_entry, b62))
        b62.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b63 = tk.Button(self, text = "k",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b63.configure(command = lambda: clickThisBtn(text_entry, b63))
        b63.place(x = keyboard_x + xx,y= keyboard_y + 100) 
        b64 = tk.Button(self, text = "&",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b64.configure(command = lambda: clickThisBtn(text_entry, b64))
        b64.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b65 = tk.Button(self, text = "`",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b65.configure(command = lambda: clickThisBtn(text_entry, b65))
        b65.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b66 = tk.Button(self, text = "z",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b66.configure(command = lambda: clickThisBtn(text_entry, b66))
        b66.place(x = keyboard_x + xx,y= keyboard_y) 
        b67 = tk.Button(self, text = "o",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b67.configure(command = lambda: clickThisBtn(text_entry, b67))
        b67.place(x = keyboard_x + xx, y = keyboard_y + 50)   
        b68 = tk.Button(self, text = "_",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b68.configure(command = lambda: clickThisBtn(text_entry, b68))
        b68.place(x = keyboard_x + xx, y = keyboard_y + 100) 
        b69 = tk.Button(self, text = "{",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b69.configure(command = lambda: clickThisBtn(text_entry, b69))
        b69.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b70 = tk.Button(self, text = "~",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b70.configure(command = lambda: clickThisBtn(text_entry, b70))
        b70.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b71 = tk.Button(self, text = "l",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b71.configure(command = lambda: clickThisBtn(text_entry, b71))
        b71.place(x = keyboard_x + xx,y= keyboard_y ) 
        b72 = tk.Button(self, text = "r",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b72.configure(command = lambda: clickThisBtn(text_entry, b72))
        b72.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b73 = tk.Button(self, text = "x",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b73.configure(command = lambda: clickThisBtn(text_entry, b73))
        b73.place(x = keyboard_x + xx, y = keyboard_y + 100)   
        b74 = tk.Button(self, text = "}",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b74.configure(command = lambda: clickThisBtn(text_entry, b74))
        b74.place(x = keyboard_x + xx,y= keyboard_y + 150) 
        b75 = tk.Button(self, text = "|",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b75.configure(command = lambda: clickThisBtn(text_entry, b75))
        b75.place(x = keyboard_x + xx,y= keyboard_y + 200) 
        
        xx += d
        
        b76 = tk.Button(self, text = "The",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b76.configure(command = lambda: clickThisBtn(text_entry, b76))
        b76.place(x = keyboard_x + xx,y= keyboard_y) 
        b77 = tk.Button(self, text = "You",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b77.configure(command = lambda: clickThisBtn(text_entry, b77))
        b77.place(x = keyboard_x + xx,y= keyboard_y + 50) 
        b78 = tk.Button(self, text = "I",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b78.configure(command = lambda: clickThisBtn(text_entry, b78))
        b78.place(x = keyboard_x + xx,y= keyboard_y + 100)
        b79 = tk.Button(self, text = "This",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b79.configure(command = lambda: clickThisBtn(text_entry, b78))
        b79.place(x = keyboard_x + xx,y= keyboard_y + 150)
        b80 = tk.Button(self, text = "So",width = kw, pady = 5,bg="#f7f7f7",relief = "sunken")
        b80.configure(command = lambda: clickThisBtn(text_entry, b78))
        b80.place(x = keyboard_x + xx,y= keyboard_y + 200)
        
        
        col1 = [b1, b2, b3, b4, b5 ]
        col2 = [b6, b7, b8, b9, b10]
        col3 = [ b11, b12, b13, b14, b15]
        col4 = [b16, b17, b18, b19, b20]
        col5 = [b21, b22, b23, b24, b25]
        col6 = [b26, b27, b28, b29, b30]
        col7 = [b31, b32, b33, b34, b35]
        col8 = [b36, b37, b38, b39, b40]
        col9 = [b41, b42, b43, b44, b45]
        col10 = [b46, b47, b48,b49, b50]
        col11 = [b51, b52, b53, b54, b55]
        col12 = [b56, b57, b58, b59, b60]
        col13 = [b61, b62, b63, b64, b65] 
        col14 = [b66,b67, b68, b69, b70]
        col15 = [b71, b72, b73, b74, b75]
        col16 = [b76, b77 , b78, b79, b80]
        
        cols = [col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12, col13, col14, col15, col16]  
        
        cap = cv2.VideoCapture(0)
#         frames_count = 0
        
        
        def my_mainloop():
            _, frame = cap.read(0)
            self.frames_count_1 += 1
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Face detection
            faces = detector(gray)
            temp_gaze = ""

            for face in faces:
                landmarks = predictor(gray, face)

                left_eye, right_eye = eyes_contour_points(landmarks)

                 # Eyes color
                cv2.polylines(frame, [left_eye], True, (0, 0, 255), 2)
                cv2.polylines(frame, [right_eye], True, (0, 0, 255), 2)

                # Detect blinking
                left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
                right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
                self.blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
                
                
                gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks, frame, gray)
                gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks, frame, gray)
                self.gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2

                if(self.blinking_ratio > 5):
                    cv2.putText(frame, "Blinking", (150, 100), font, 2, (0, 255 ,0), 1)
                else:
                    if self.gaze_ratio <= 0.9:
                        temp_gaze = "RIGHT"
                        cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0, 0, 255), 3)
                    elif 1 < self.gaze_ratio < 1.7:
                        temp_gaze = "CENTER"
                        cv2.putText(frame, "CENTER", (50, 100), font, 2, (0, 0, 255), 3)
                    else:
                        temp_gaze = "LEFT"
                        cv2.putText(frame, "LEFT", (50, 100), font, 2, (0, 0, 255), 3)
                        
            text_entry_1.configure(state='normal')
            text_entry_1.delete(0,'end')
            text_entry_1.insert(0, temp_gaze)
            text_entry_1.configure(state='disabled') 
            
#             highlightCol(cols, 2)
                        
            if (self.key_gaze):                
                if(self.gaze_ratio <= 0.9):
                    self.blinking_frames_ri += 1
                    self.frames_count_1 -= 1

                    if(self.blinking_frames_ri == self.active_gaze):
                        self.key_blink = True
                        self.key_gaze = False
                        self.frames_count_1 = 0
                        self.gaze_sight = "Right"
                        self.blinking_frames_ri = self.blinking_frames_le = 0 

                elif (self.gaze_ratio >= 1.7):
                    self.blinking_frames_le += 1
                    self.frames_count_1 -= 1

                    if(self.blinking_frames_le == self.active_gaze):
                        self.key_blink = True
                        self.key_gaze = False
                        self.frames_count_1 = 0
                        self.gaze_sight = "Left"
                        self.blinking_frames_le = self.blinking_frames_ri = 0

            if(self.key_blink):
                if(self.gaze_sight == "Left"):
                    highlightCol(cols, self.index_letter)

                    if(self.blinking_ratio > 4.9):
                        self.blinking_frames += 1
                        self.frames_count_1 -= 1

                        if(self.blinking_frames == self.active_blinking):
                            self.key_row = True
                            self.key_blink = False
                            self.frames_count_1 = 0
                            self.blinking_frames = 0
                    else:
                        self.blinking_frames = 0
                        

                    if(self.frames_count_1 == self.frame_limit):
                        self.index_letter += 1
                        self.frames_count_1 = 0
                    if(self.index_letter == 8):
                        self.index_letter = 0
                else:
                    highlightCol(cols, 8 + self.index_letter)
                    if(self.blinking_ratio > 4.9):
                        self.blinking_frames += 1
                        self.frames_count_1 -= 1

                        if(self.blinking_frames == self.active_blinking):
                            self.key_row = True
                            self.key_blink = False
                            self.frames_count_1 = 0
                            self.blinking_frames = 0
                            self.blinking_frames_le = self.blinking_frames_ri = 0
                    else:
                        self.blinking_frames = 0
                            

                    if(self.frames_count_1 == self.frame_limit):
                        self.index_letter += 1
                        self.frames_count_1 = 0
                    if(self.index_letter == 8):
                        self.index_letter = 0

            if (self.key_row):
                row_number= self.index_letter
                if (self.gaze_sight == "Right"):
                    highlightBtn(cols, 8 + self.index_letter, self.col_letter)
                    row_number += 8

                else:
                    highlightBtn(cols, self.index_letter, self.col_letter)

                if(self.frames_count_1 == self.frame_limit):
                    self.col_letter += 1
                    self.frames_count_1 = 0
                if(self.col_letter == 5):
                    self.col_letter = 0

                if(self.blinking_ratio > 4.9):
                        self.blinking_frames += 1
                        self.frames_count_1 -= 1

                        if(self.blinking_frames == self.active_blinking):
                            self.key_row = False
                            self.text += get_letter_key(cols,row_number,self.col_letter)
                            print(self.text)
                            clickBtn(text_entry, cols,row_number,self.col_letter)

                            self.key_blink = False
                            self.key_gaze = True
                            self.frames_count_1 = 0
                            self.blinking_frames = 0 
                            self.blinking_frames_le = self.blinking_frames_ri = 0

                            self.col_letter = 0
                            self.index_letter = 0


            self.after(10, my_mainloop)


        self.after(10,my_mainloop)


In [19]:
import time

start = time.time()

window = tkinterApp()
window.mainloop()

end = time.time()

b
bu
buq
buqbuses


In [20]:
print(end - start)

63.06331205368042
